# Webhook Configuration

* Webhooks are supported for the following events:
    1. label_created
    2. label_updated
    3. label_deleted
    4. review_created
    5. review_updated
    6. review_deleted

In [4]:
from flask import Flask, request
import json
import hmac
import hashlib
import threading
from werkzeug.serving import run_simple
import os
from labelbox import Client, Webhook
import requests

In [5]:
client = Client()

In [6]:
#Set this to a project that you want to get updates on.
PROJECT_ID = "ckm4xyfncfgja0760vpfdxoro"

### Configure NGROK (Optional)
* If you do not have a public ip address then follow along

1. Create an account:
    https://dashboard.ngrok.com/get-started/setup
2. Download ngrok and extract the zip file
3. Add ngrok to your path
4. Add the authtoken `ngrok authtoken <token>`

In [7]:
#We are using port 3001 for this example.
#Feel free to set to whatever port you want
WH_PORT = 3001
os.system(f'ngrok http {WH_PORT} &')
#WARNING: Make sure you clean this up at the end!!
###

0

### Configure server to receive requests

In [8]:
# This can be any secret that matches your webhook config (we will set later)
secret = b'example_secret'

In [9]:

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'


@app.route('/webhook-endpoint', methods=['POST'])
def print_webhook_info():
    payload = request.data
    computed_signature = hmac.new(secret, msg=payload, digestmod=hashlib.sha1).hexdigest()
    if request.headers['X-Hub-Signature'] != 'sha1='+computed_signature:
        print('Error: computed_signature does not match signature provided in the headers')
        return 'Error', 500, 200

    print('=========== New Webhook Delivery ============')
    print('Delivery ID: %s' % request.headers['X-Labelbox-Id'])
    print('Event: %s' % request.headers['X-Labelbox-Event'])
    print('Payload: %s' % json.dumps(json.loads(payload.decode('utf8')),indent=4))
    return 'Success'

thread = threading.Thread(target=lambda: run_simple('0.0.0.0', WH_PORT, app))
thread.start()

#### Test server

In [10]:
print(requests.get('http://localhost:3001').text)

 * Running on http://0.0.0.0:3001/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Mar/2021 09:51:50] "GET / HTTP/1.1" 200 -


Hello, World!


### Create Webhook

- Set ip address if your ip is publicly accessible.
- Otherwise use the following to get ngrok public_url

In [11]:
res = requests.get('http://localhost:4040/api/tunnels')
assert res.status_code == 200, f"ngrok probably isn't running. {res.status_code}, {res.text}"
tunnels = res.json()['tunnels']
tunnel = [t for t in tunnels if t['config']['addr'].split(':')[-1] == str(WH_PORT)]
tunnel = tunnel[0] #Should only be one..
public_url = tunnel['public_url']

In [12]:
#Set project to limit the scope to a single project
project = client.get_project(PROJECT_ID)
topics = {topic.value for topic in Webhook.Topic}
webhook = Webhook.create(client, topics = topics, url = public_url, secret = secret.decode(), project = project)

In [13]:
#Ok so we should be configured assuming everything is setup correctly.
#Go to the following url and make a new label to see if it works
print(f"https://app.labelbox.com/projects/{PROJECT_ID}")

https://app.labelbox.com/projects/ckm4xyfncfgja0760vpfdxoro


In [14]:
## Oops we made a mistake. The url was incorrect. We pointed the url to the hello world endpoint.

### Update Webhook

In [15]:
#url, topics, and status can all be updated
updated_url = f"{public_url}/webhook-endpoint"
print(updated_url)
webhook.update(url = updated_url)

https://eb3e7f8eb089.ngrok.io/webhook-endpoint


127.0.0.1 - - [17/Mar/2021 09:52:00] "POST /webhook-endpoint HTTP/1.1" 200 -


=========== New Webhook Delivery ============
Delivery ID: ckmdi74ew57lq063553kuzekq
Event: LABEL_UPDATED
Payload: {
    "id": "ckmb8h50t008f3h683tugqsap",
    "createdAt": "2021-03-15T23:44:18Z",
    "updatedAt": "2021-03-17T13:51:59Z",
    "secondsToLabel": 79.173,
    "label": "{\"objects\":[{\"featureId\":\"ckmb8gvd9008b3h68l5tcmrt1\",\"schemaId\":\"ckm3se1a5010y0y6rcdc0bvvl\",\"title\":\"animal\",\"value\":\"animal\",\"color\":\"#1CE6FF\",\"bbox\":{\"top\":325,\"left\":233,\"height\":266,\"width\":413},\"instanceURI\":\"https://api.labelbox.com/masks/feature/ckmb8gvd9008b3h68l5tcmrt1\"},{\"featureId\":\"ckmbdp94300013h68y14zluto\",\"schemaId\":\"ckmbdol5s1ybk0y96fje86ktu\",\"title\":\"human\",\"value\":\"human\",\"color\":\"#FF34FF\",\"bbox\":{\"top\":75,\"left\":527,\"height\":453,\"width\":155},\"instanceURI\":\"https://api.labelbox.com/masks/feature/ckmbdp94300013h68y14zluto\"},{\"featureId\":\"ckmbdpdda00043h688pqrsoku\",\"schemaId\":\"ckmbdol5s1ybk0y96fje86ktu\",\"title\":\"h

### List and delete all webhooks

In [17]:
#DELETE:
webhook.update(status = Webhook.Status.INACTIVE.value)

#FETCH ALL WEBHOOKS:
org = client.get_organization()
webhooks = org.webhooks()


#Run this to clear all.
#WARNING!!! THIS WILL DELETE ALL WEBHOOKS FOR YOUR ORG
#ONLY RUN THIS IS YOU KNOW WHAT YOU ARE DOING.
#for webhook in webhooks:
#    print(webhook)
#    webhook.update(status = Webhook.Status.INACTIVE.value)

In [18]:
#Cleanup:
!pkill -9 ngrok